## Imports & Constants

Here we just define some imports and select some test images to see later how well our models perform.

In [1]:
import os
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.utils import image_dataset_from_directory

# For reproducability
tf.random.set_seed(0)
tf.config.experimental.enable_op_determinism()

# For classification, any dataset works under anylgasses (face-attributes-grouped is just example)
CLS_DATA_ROOT = "/kaggle/input/glasses-detector/classification/anyglasses/face-attributes-grouped/"
SEG_DATA_ROOT = "/kaggle/input/face-synthetics-glasses/face-synthetics-glasses/"
AUTOTUNE = tf.data.AUTOTUNE

# Classification test paths
TEST_IMG_PATHS_CLS = [
    os.path.join(CLS_DATA_ROOT, "test/no_anyglasses/0f57d9216de31fd27dcac57c61a61019.jpg"),
    os.path.join(CLS_DATA_ROOT, "test/anyglasses/4.jpg"),
    os.path.join(CLS_DATA_ROOT, "test/no_anyglasses/000031c9.jpg"),
    os.path.join(CLS_DATA_ROOT, "test/anyglasses/54ca1e397e9f765801ccbbf33f41c808.jpg"),
    os.path.join(CLS_DATA_ROOT, "test/no_anyglasses/1438659549.jpg"),
]

# Segmentation test paths
TEST_IMG_PATHS_SEG = [
    os.path.join(SEG_DATA_ROOT, "test/images/000410.jpg"),
    os.path.join(SEG_DATA_ROOT, "test/images/000664.jpg"),
    os.path.join(SEG_DATA_ROOT, "test/images/001034.jpg"),
    os.path.join(SEG_DATA_ROOT, "test/images/000987.jpg"),
    os.path.join(SEG_DATA_ROOT, "test/images/001421.jpg"),
]

2024-04-01 15:41:24.468510: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-01 15:41:24.468668: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-01 15:41:24.602493: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Utilities

Here we define a custom **F1Score** metric which passes the predictions through sigmoid first, two custom methods for loading classification and segmentation data and a method for displaying classifier or segmenter predictions.

In [2]:
class SigmoidF1Score(tf.keras.metrics.F1Score):
    def update_state(self, y_true, y_pred, sample_weight=None):
        # Flatten across batch; squeeze preds to [0, 1]
        shape = (-1, tf.shape(y_true)[-1])
        y_true = tf.reshape(y_true, shape)
        y_pred = tf.reshape(y_pred, shape)
        y_pred = tf.nn.sigmoid(y_pred)

        return super().update_state(y_true, y_pred, sample_weight)

def create_dataset_cls(split):
    # Load the dataset from directory
    ds = image_dataset_from_directory(
        directory=os.path.join(CLS_DATA_ROOT, split),
        label_mode="binary",
        shuffle=split=="train",
        seed=0,
    )
    class_names = ds.class_names
    
    # Specify buffer size and image rescaling
    ds = ds.cache().prefetch(buffer_size=AUTOTUNE)
    ds = ds.map(lambda x, y: (layers.Rescaling(1./255)(x), y))
    ds.class_names = class_names
    
    return ds

def create_dataset_seg(split):
    # Load the image dataset from directory
    ds_img = image_dataset_from_directory(
        directory=os.path.join(SEG_DATA_ROOT, split, "images"),
        label_mode=None,
        shuffle=split=="train",
        seed=0,
    )
    
    # Load the mask dataset from directory
    ds_msk = image_dataset_from_directory(
        directory=os.path.join(SEG_DATA_ROOT, split, "masks"),
        label_mode=None,
        color_mode="grayscale",
        shuffle=split=="train",
        seed=0,
    )
    
    # Join and specify buffer and rescaling
    ds = tf.data.Dataset.zip((ds_img, ds_msk))
    ds = ds.map(lambda x, y: (layers.Rescaling(1./255)(x), tf.where(y > 127, 1., 0.)))

    return ds

def display_predictions(ds, model, task, num_images=5, seed=0):
    
    if task == "classification":
        # If the task is classification
        img_paths = TEST_IMG_PATHS_CLS
    elif task == "segmentation":
        # If the task is segmentation
        img_paths = TEST_IMG_PATHS_SEG
    
    # Load images from paths
    images = np.array([
        tf.keras.utils.img_to_array(tf.keras.utils.load_img(img_path))
        for img_path in img_paths
    ])
    
    # Predict with model & initialize plot
    predictions = model.predict((1/255) * images)
    plt.figure(figsize=(20, 4))
    
    for i in range(num_images):
        # Plot the input image in ith column
        ax = plt.subplot(1, num_images, i + 1)
        plt.imshow(images[i].astype("uint8"))
        plt.axis("off")
        
        if task == "classification":
            # Display the prediction as an image title
            plt.title(ds.class_names[int(predictions[i][0] > 0)])
        elif task == "segmentation":
            # Display the prediction as overlaid image
            colors = [(0, 0, 0, 0)] + [(1, 0, 0, 1)] * 255
            cm = mcolors.LinearSegmentedColormap.from_list("red", colors, N=256)
            plt.imshow(1 * (predictions[i].squeeze() > 0), alpha=0.5, cmap=cm, vmin=0, vmax=1)
    
    # Show preds
    plt.show()

## Classification

We define a simple classifier, train for 5 epochs, and see the results.

## Segmentation

We define a simple segmenter, train for 5 epochs, and see the results (could be better if the model was more complex).

In [ ]:
# Generate datasets from image directories
ds_seg_train = create_dataset_seg("train")
ds_seg_val = create_dataset_seg("val")

# Very simple segmenter
segmenter = Sequential([
    layers.Conv2D(50, 3, padding="same", activation="relu"),
    layers.BatchNormalization(),
    layers.Conv2D(100, 3, padding="same", activation="relu"),
    layers.BatchNormalization(),
    layers.Conv2D(150, 3, padding="same", activation="relu"),
    layers.BatchNormalization(),
    layers.Conv2D(100, 3, padding="same", activation="relu"),
    layers.BatchNormalization(),
    layers.Conv2D(50, 3, padding="same"),
    layers.BatchNormalization(),
    layers.Conv2D(1, 1),
])

# Compile with AdamW
segmenter.compile(
    optimizer=tf.keras.optimizers.AdamW(learning_rate=0.0001),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[SigmoidF1Score(average="micro", threshold=.5, name="dice")],
)

# Fit the model and show some predictions from selected test images
segmenter.fit(ds_seg_train, validation_data=ds_seg_val, epochs=5)
display_predictions(ds_cls_val, segmenter, task="segmentation")

Found 11372 files belonging to 1 classes.
Found 11372 files belonging to 1 classes.
Found 1481 files belonging to 1 classes.
Found 1481 files belonging to 1 classes.
Epoch 1/5


I0000 00:00:1711986183.265100     114 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


168/356 [=============>................] - ETA: 1:48 - loss: 0.6501 - dice: 0.2187